<a href="https://colab.research.google.com/github/gsjunqueira/ONS/blob/main/Gera%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Geração dos 10 maiores geradores do Brasil***

Desenvolvido por: ***Giovani Santiago Junqueira***

##<font color = lightblue>Fonte de dados


***1)*** <font color = gray> para os anos de 2000 a 2021 os dados de geração foram conseguidos através do **<font color=black>Dados Abertos do ONS** <font color = gray> através da URL: https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_ho/GERACAO_USINA_{ano}.csv

***2)*** <font color = gray> para os anos a partir de 2022 os dados de geração foram conseguidos através do **<font color=black>Dados Abertos do ONS** <font color = gray> através da URL: https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA-2_{ano}_{mes}.csv



###GERAÇÃO POR USINA EM BASE HORÁRIA
<font color = gray>Geração verificada de usinas, conjuntos de usinas e grupos de pequenas usinas em base horária. Os Conjuntos são formados por usinas classificadas na modalidade Tipo II-C, conforme Submódulo 7.2 dos Procedimentos de Rede e são estabelecidos em Ajustamentos Operativos, disponíveis no MPO. Os grupos de pequenas usinas são formados por usinas Tipo III, que não possuem relacionamento com o ONS, e os dados são referentes a previsões de geração. Os dados dos anos de 2000 a 2021 estão disponibilizados em arquivos agrupados pelos respectivos anos, e a partir do ano de 2022 as informações estão agrupados em arquivos por mês/ano. Os dados disponibilizados fazem parte de um processo de consistência recorrente e, portanto, podem ser atualizados após a sua publicação.
###Aviso Legal
<font color = gray>Ao acessar este site e/ou utilizar as informações provenientes dele, será considerado que você aceitou os termos e condições da LICENÇA CC-BY, que permite que os reutilizadores distribuam, modifiquem, adaptem e desenvolvam o material sobre os dados, desde que seja dado o crédito apropriado ao criador(ONS) e que informe quais alterações foram feitas. Os dados são fornecidos "como estão" e apenas para fins informativos; e este conjunto de dados pode ter sua disponibilidade interrompida a qualquer momento e por qualquer motivo
###Aviso Legal: Licença
*http://opendefinition.org/od/2.1/pt-br*

##<font color = lightblue>Inicializando as bibliotecas do Python

***https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html***

In [ ]:
# coding=UTF-8

__author__ = "Giovani Santiago Junqueira"

from google.colab import drive
from google.colab import files

from datetime import datetime
from urllib import request

import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import os
plt.style.use('fivethirtyeight')
%matplotlib inline

###<font color = lightblue>Montar o Google Drive no Colab com o nome de gdrive

In [ ]:
## Montar o Google Drive no Colab com o nome de gdrive

drive.mount('/content/gdrive')

# O Comando abaixo mostra qual diretório você está trabalhando no momento.

print( os.getcwd() )

Caminho = 'gdrive/My Drive/Geração/geracao.xlsx'

##<font color = lightblue> Lista contendo os anos onde se tem o histórico de energia vertida turbinável

In [ ]:
anos = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023]

ano = anos[-1]

mes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

##<font color = lightblue>Função que realiza a leitura dos arquivos CSV e realiza a preparação dos dataframes

In [ ]:
def ler_arquivo(ano):

    geracoes = []

    dtype = {'nom_tipocombustivel' : 'category',
        'nom_tipousina' :	'category',
        'nom_usina' : 'category',
        'val_geracao' : 'float32' }

    if ano < 2022:
        request.urlretrieve(f'https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_ho/GERACAO_USINA_{ano}.csv',
                            f'GERACAO_USINA_{ano}.csv')
        geracoes.append(pd.read_csv(f'GERACAO_USINA_{ano}.csv', sep=';', dtype = dtype, encoding = "utf-8", parse_dates=['din_instante'],
                                    usecols = ['din_instante', 'nom_tipousina', 'nom_tipocombustivel', 'nom_usina', 'val_geracao']))

    elif ano < datetime.now().year:
        for j in range(len(mes)):
            request.urlretrieve(f'https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA-2_{ano}_{mes[j]}.csv',
                                f'GERACAO_USINA-2_{ano}_{mes[j]}.csv' )
            geracoes.append(pd.read_csv(f'GERACAO_USINA-2_{ano}_{mes[j]}.csv', sep=';', dtype = dtype, encoding = "utf-8", parse_dates=['din_instante'],
                                        usecols = ['din_instante', 'nom_tipousina', 'nom_tipocombustivel', 'nom_usina', 'val_geracao']))

    else:
        for j in range(len(mes)):
            if int(mes[j])< datetime.now().month:
                request.urlretrieve(f'https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA-2_{ano}_{mes[j]}.csv',
                                    f'GERACAO_USINA-2_{ano}_{mes[j]}.csv' )
                geracoes.append(pd.read_csv(f'GERACAO_USINA-2_{ano}_{mes[j]}.csv', sep=';', dtype = dtype, encoding = "utf-8", parse_dates=['din_instante'],
                                            usecols = ['din_instante', 'nom_tipousina', 'nom_tipocombustivel', 'nom_usina', 'val_geracao']))

    return pd.concat(geracoes)

##<font color = lightblue>Chamada da função que realiza a leitura dos arquivos e retorna o dataframe

In [ ]:
ger = ler_arquivo(ano)

##<font color = lightblue>Inserção de novas colunas no dataframe

In [ ]:
ger.insert(loc = 1, column = 'data', value = ger['din_instante'].dt.date)
ger.insert(loc = 1, column = 'ano', value = ger['din_instante'].dt.year)
ger.insert(loc = 1, column = 'mes', value = ger['din_instante'].dt.month)

##<font color = lightblue>Mudando o nome da usina de Itaipu 50Hz e 60Hz para somente Itaipu

In [ ]:
ger['nom_usina'].replace({'Itaipu 60 Hz': 'Itaipu', 'Itaipu 50 Hz': 'Itaipu'}, inplace = True)

##<font color = lightblue>Realizando a soma da geração das usinas

In [ ]:
geracao = ger.groupby(['nom_usina'])['val_geracao'].sum().reset_index()
geracao.sort_values(by = ['val_geracao'],  ascending=False, inplace = True)

saida = geracao.head(10)

pd.options.display.float_format = '{:.2f}'.format

##<font color = Darkblue>Saída salva em arquivo 'xlsx' no Google drive

In [ ]:
filtro = ger[ger['val_geracao']>0]['din_instante'].max()
saida.to_excel(Caminho, index = False)
filtro

##<font color = Darkblue>Saída Gráfica

In [ ]:
fig, ax = plt.subplots( figsize = (15, 5))
bar_colors = ['indigo', 'darkviolet', 'darkmagenta', 'darkred', 'red', 'orangered', 'darkorange', 'orange', 'gold', 'yellow']

valores = saida['val_geracao'].tolist()
usinas = saida['nom_usina'].tolist()

ax.bar(usinas, valores, color=bar_colors)
ax.set_ylabel('Energia Gerada (MWh)')
ax.xaxis.set_tick_params(rotation=75)
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
ax.set_title(f'Valores de energia gerada para o ano de {ano}')
plt.show()